**<font size=4>This is my advanced study of the first competition I attended on Kaggle</font>**


**<font size=2>I believe that everybody is not unfamliar with the Titanic</font>**



![picture](http://www.oscars.org/sites/oscars/files/1083_dpk02718_04_t3d-4k-004.jpg)

As a data scientist, I think we should all be aware of the ["Black Swam"](https://en.wikipedia.org/wiki/Black_swan_theory)

**Those things that we can hardly predict**

To make this world better, I think we should work hard to make right decisions based on the analysis and prediction, but meanwhile, be aware of the probability of happening the so-called Black Swam.
Hope the tragedy like this will never happen again.

**<font size=5>Now, let's back to the topic</font>**


**At first, I think this is a great chance to have a thorough look at my foundation of data science skills**

Therefore, first, I want to have my plan of this case here.
* [Dataset Overlook](#1)
* [Data Imputation](#2)
* [Exploratory Data Analysis (EDA)](#3)
* [Features Engineering & Data Munging](#4)
* [Modeling](#5)
* [Validation](#6)
* [Explain the model](#7)

## **<a id='1'>Overlook</a>**


**First, input data and modules and take a look at it**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from string import ascii_letters
import seaborn as sns

plt.style.use('ggplot')

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
train.head()

**<font size=4>To be more precise, we need to understand the meanings of each columns.</font>**


**Variable Notes**

**pclass**: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...

*Sibling* = brother, sister, stepbrother, stepsister

*Spouse* = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...

*Parent* = mother, father

*Child* = daughter, son, stepdaughter, stepson

Some children travelled only with a nanny, therefore parch=0 for them.

**About PassengerId & Ticket, obviously, PassengerId is just the order number of each row, and I think Ticket is just a random string of each ticket.**

In [ ]:
# Drop ID as it won't indicate too many things here
train.drop(['PassengerId'],axis=1,inplace=True)

Now, I want to see how many null values in both data

In [ ]:
pd.isnull(train).sum()

In [ ]:
pd.isnull(test).sum()

Most of the missing values are in Age and Cabin columns

Now, I want to have a basic understand of how the data distribute

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
for i in train.select_dtypes(include=['object']):
    print('Column ',i,' has ',train[i].nunique(),' unique values')

In [ ]:
for i in test.select_dtypes(include=['object']):
    print('Column ',i,' has ',test[i].nunique(),' unique values')

## **<a id='2'>Data Imputation</a>**

Here I put training set and testing set together so that I can do preprocess at the same time and after data imputation I copy a set of training set so that I can do EDA with it.

In [ ]:
target = train.Survived
train_df = train.drop('Survived',axis=1)
test_df = test.drop('PassengerId',axis=1)
train_df['is_train'] = 1
test_df['is_train'] = 0
train_test = pd.concat([train_df,test_df],axis=0)

**A smart way to impute Age !!!** Refer to : https://www.kaggle.com/ash316/eda-to-prediction-dietanic/notebook

In [ ]:
train_test['Initial']=0
for i in train_test:
    train_test['Initial']=train_test.Name.str.extract('([A-Za-z]+)\.')
pd.crosstab(train_test.Initial,train_test.Sex).T.style.background_gradient(cmap='summer_r')

In [ ]:
train['Initial']=0
for i in train:
    train['Initial']=train.Name.str.extract('([A-Za-z]+)\.')
pd.crosstab(train.Initial,train.Sex).T.style.background_gradient(cmap='summer_r')

In [ ]:
pd.crosstab(train.Initial,train.Survived).T.style.background_gradient(cmap='summer_r')

In [ ]:
train_test['Initial'].replace(['Capt','Col','Countess','Don','Dona','Dr','Jonkheer','Lady','Major','Master','Mlle','Mme','Ms','Rev','Sir'],
                            ['Special_male','Other_male','Special','Special_male','Special_female','Other','Special_male','Special','Special_male','Other_male','Special','Special','Special','Other_male','Special'],inplace=True)

Here I look deeper to make sure there might be some special guests with special title

In [ ]:
train_test.groupby(['Initial','Pclass'])['Age'].aggregate(['mean','count'])

In [ ]:
train_test.loc[(train_test.Age.isnull())]['Initial'].value_counts()

In [ ]:
train_test.loc[(train_test.Age.isnull())]['Pclass'].value_counts()

In [ ]:
train_test.loc[(train_test.Age.isnull())].groupby(['Initial','Pclass'])['Name'].aggregate('count')

In [ ]:
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
temp_train_test = pd.get_dummies(train_test[['Initial','Pclass','Age']])
train_test_age_filled = imp.fit_transform(temp_train_test)

In [ ]:
train_test_age_filled = pd.DataFrame(train_test_age_filled,columns=temp_train_test.columns.tolist())

In [ ]:
train_test_age_filled.head()

In [ ]:
train_test.Age = train_test_age_filled.Age

In [ ]:
train_test.head()

In [ ]:
train_test[np.isnan(train_test['Fare'])]

In [ ]:
corr = train_test.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, xticklabels=corr.columns.values,yticklabels=corr.columns.values,mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .5})

I tell the missing fare by the PClass, Parch, and SibSp.

In [ ]:
train_test.groupby(['Pclass','Parch','SibSp'])['Fare'].mean()

In [ ]:
train_test.loc[(train_test.Fare.isnull()),'Fare']=9

In [ ]:
sns.countplot(train_test.Embarked)

In [ ]:
train_test.loc[train_test.Embarked.isnull()]

We can see that these two held the same ticket. Therefore, the other columns are almost the same!

In [ ]:
train_test_temp = pd.get_dummies(train_test[['Fare','Parch','Pclass','Embarked']])

corr = train_test_temp.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, xticklabels=corr.columns.values,yticklabels=corr.columns.values,mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .5})

The embarked is probably decided by or correlated to Fare and Pclass

In [ ]:
train_test.groupby(['Embarked','Pclass'])['Fare'].mean()

In [ ]:
train_test["Embarked"] = train_test["Embarked"].fillna("S")

### Keep Cabin and impute Cabin or not

In [ ]:
from sklearn.feature_selection import mutual_info_classif

train_temp = pd.get_dummies(train['Cabin'])

res = dict(zip(train_temp.columns.tolist(),
               mutual_info_classif(train_temp, train['Survived'], discrete_features=True)
               ))
print(res)

It seems that Cabin won't offer too much useful information for the prediction. Thus, I'll drop it.

In [ ]:
train_test.drop('Cabin',axis=1,inplace=True)

Finally, make sure there is no more NA value

In [ ]:
pd.isnull(train_test).sum()

In [ ]:
train_df = train_test[train_test.is_train == 1]

## **<a id='3'>EDA</a>**

In [ ]:
train_df.drop(['is_train'],axis=1,inplace=True)

In [ ]:
train_df['Survived'] = train['Survived']

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
corr = train_df.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, xticklabels=corr.columns.values,yticklabels=corr.columns.values,mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .5})

In [ ]:
f,ax=plt.subplots(2,3,figsize=(12,16))
sns.countplot('Pclass',data=train_df,ax=ax[0,0],hue='Survived')
ax[0,0].set_title('Pclass distribution')
sns.countplot('Sex',data=train_df,ax=ax[0,1],hue='Survived')
ax[0,1].set_title('Sex distribution')
sns.countplot('Initial',data=train_df,ax=ax[0,2],hue='Survived').set_xticklabels(rotation=40,labels=['Mr','Mrs','Miss','Other_male','Special_male','Other','Special'])
ax[0,2].set_title('Sex distribution')
sns.countplot('SibSp',data=train_df,ax=ax[1,0],hue='Survived')
ax[1,0].set_title('SibSp distribution')
sns.countplot('Parch',data=train_df,ax=ax[1,1],hue='Survived')
ax[1,1].set_title('Parch distribution')
sns.countplot('Embarked',data=train_df,ax=ax[1,2],hue='Survived')
ax[1,2].set_title('Embarked distribution')

This is a more thorough look at how data distribute and how they interact with survived or not

**And now, let's dig deeper with whether these people survived or not**

In [ ]:
pd.crosstab(train_df.Pclass,train_df.Survived).apply(lambda r: r/r.sum(), axis=1).style.background_gradient(cmap='summer_r')

About 2 thirds of the people in PClass1 survived and only half of the people in PClass3 survived.
I'm considering treating PClass as numbers or dummy variable. But now, I'll just leave it like this.

In [ ]:
plt.title("Age & Survival Distribution")
plt.hist([train_df[train_df['Survived']==1]['Age'],train_df[train_df['Survived']==0]['Age']],bins = 10,label=['Survived', 'Dead'])
plt.legend()
plt.show()

In terms of age, most of the people under 40 survived. And the ratio gets worse by the age gets older.

Let's separate these people into groups based on age.

In [ ]:
train_df['Age_Range']=pd.qcut(train_df['Age'],7)
train_df.groupby(['Age_Range'])['Survived'].mean().to_frame().style.background_gradient(cmap='summer_r')

In [ ]:
plt.title("Age & Survival Distribution")
plt.hist([train_df[train_df['Survived']==1]['Age'],train_df[train_df['Survived']==0]['Age']],bins = 25,label=['Survived', 'Dead'])
plt.legend()
plt.show()

Here we can see that people aged 29 are less likely to survive.

Mayber we can separate them into males and females to look deeper.

In [ ]:
sns.factorplot('Age_Range','Survived',hue='Sex',data=train_df)
plt.xticks(rotation=45)

Obviously, most groups of female are more likely to survive. And boys are more likely to survive than male adults.

In [ ]:
plt.title("SibSp & Survival Distribution")
plt.hist([train_df[train_df['Survived']==1]['SibSp'],train_df[train_df['Survived']==0]['SibSp']],bins=10,range=[0,9],label=['Survived', 'Dead'])
plt.legend()
plt.show()

In [ ]:
pd.crosstab(train_df.SibSp,train_df.Survived).apply(lambda r: r/r.sum(), axis=1).style.background_gradient(cmap='summer_r')

Well, I insist if we input thise feature without any preprocessing, it will cause over-fitting since people with 5 and 8 SibSp all survived, but there aren't many of them.

In [ ]:
plt.title("Parch & Survival Distribution")
plt.hist([train_df[train_df['Survived']==1]['Parch'],train_df[train_df['Survived']==0]['Parch']],bins=7,range=[0,7],label=['Survived', 'Dead'])
plt.legend()
plt.show()

In [ ]:
pd.crosstab(train_df.Parch,train_df.Survived).apply(lambda r: r/r.sum(), axis=1).style.background_gradient(cmap='summer_r')

This as well.

In [ ]:
plt.title("Fare & Survival Distribution")
plt.hist([train_df[train_df['Survived']==1]['Fare'],train_df[train_df['Survived']==0]['Fare']],bins=15,label=['Survived', 'Dead'])
plt.legend()
plt.show()

In [ ]:
train_df['Fare_Range']=pd.qcut(train_df['Fare'],12)
train_df.groupby(['Fare_Range'])['Survived'].mean().to_frame().style.background_gradient(cmap='summer_r')

In [ ]:
train_df['Name_len'] = train_df.Name.apply(lambda x: len(x))

In [ ]:
plt.title("Length of name & Survival Distribution")
plt.hist([train_df[train_df['Survived']==1]['Name_len'],train_df[train_df['Survived']==0]['Name_len']],bins=15,label=['Survived', 'Dead'])
plt.legend()
plt.show()

In [ ]:
train_df['NameLen_Range']=pd.qcut(train_df['Name_len'],12)
train_df.groupby(['NameLen_Range'])['Survived'].mean().to_frame().style.background_gradient(cmap='summer_r')

Though this might sounds weird, people with longer name seem to be more likely to survive.

(Maybe from a rich family)

In [ ]:
sns.factorplot('NameLen_Range','Survived',hue='Pclass',data=train_df)
plt.xticks(rotation=45)

In [ ]:
sns.countplot('NameLen_Range',hue='Survived',data=train_df)
plt.xticks(rotation=45)

Basically, except people have name length between 19 and 20 all survived, name length and Pclass have positive correlation with the survival

In [ ]:
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch']

In [ ]:
train_df.FamilySize.describe()

In [ ]:
plt.title("FamilySize & Survival Distribution")
plt.hist([train_df[train_df['Survived']==1]['FamilySize'],train_df[train_df['Survived']==0]['FamilySize']],range(0,11),label=['Survived', 'Dead'])
plt.legend()
plt.show()

In [ ]:
pd.crosstab(train_df.FamilySize,train_df.Survived).apply(lambda r: r/r.sum(), axis=1).style.background_gradient(cmap='summer_r')

Except those with more than 4 family members, basically, the bigger the family size the more likely to survive. However, I'll split them into alone, small family, and big family.

In [ ]:
sns.countplot('Sex',hue='Survived',data=train_df)

In [ ]:
pd.crosstab(train_df.Sex,train_df.Survived).apply(lambda r: r/r.sum(), axis=1).style.background_gradient(cmap='summer_r')

Without a doubt, the proportion of women survived are much higher than that of men

In [ ]:
sns.factorplot('Pclass','Survived',hue='Sex',data=train_df)

It seems not matter which Pclass it was, the people tried their best to save most of the women.

In [ ]:
sns.factorplot('Fare_Range','Survived',hue='Pclass',data=train_df)
plt.xticks(rotation=45)

This is pretty straightforward.

Those with higher fare are not more likely to survive.
But we can see that those with Pclass 1, must have a fare over 20.5. 
And those people in Pclass 1 and fare 52 and 80 rarely survived.
Also, there is some people in Pclass 2 with Fare 0. I need to investigate that.

In [ ]:
train_df.loc[(train_df['Pclass']==2) & (train_df['Fare'] == 0)]

It seems these tickets are mostly bought together or something. However, since there are 6 of them, I'll keep them as they are now.

**<font size=5>Features Engineering & Data Munging</font>**

In [ ]:
train_test['Last_Name'] = train_test.Name.apply(lambda x: x.split(' (')[0].split('"')[0].split(' ')[-1])

In [ ]:
train_test.head()

In [ ]:
train_test['NameLen'] = train_test.Name.apply(lambda x : len(x))

In [ ]:
train_test['Sex'] = train_test['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [ ]:
train_test['FamilySize'] = train_test['Parch'] + train_test['SibSp']

In [ ]:
train_test['Alone']=0
train_test.loc[(train_test.FamilySize==0),'Alone']=1

In [ ]:
train_test['Small_Family']=0
train_test.loc[(train_test.FamilySize<=3)&(train_test.FamilySize!=0),'Small_Family']=1

In [ ]:
train_test['Big_Family']=0
train_test.loc[(train_test.FamilySize>=4),'Big_Family']=1

In [ ]:
train_test.head()

In [ ]:
train_test.drop(['Name','SibSp','Parch','Ticket','FamilySize'],axis=1,inplace=True)

In [ ]:
train_test = pd.get_dummies(train_test)

In [ ]:
train_test.head()

In [ ]:
train_test.info()

**<font size=5>Modeling</font>**

In [ ]:
train = train_test[train_test.is_train == 1].drop(['is_train'],axis=1)

test = train_test[train_test.is_train == 0].drop(['is_train'],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(train,train_df['Survived'],test_size=0.33,random_state=3)

In [ ]:
model = LogisticRegression(random_state=51)
model.fit(X_train, Y_train)
prediction=model.predict(X_test)
print('Accuracy for rbf LogisticRegression is ',np.mean(cross_val_score(model, train, train_df['Survived'], cv=3)))

In [ ]:
sns.heatmap(confusion_matrix(prediction,Y_test),annot=True,fmt='2.0f')

In [ ]:
model = RandomForestClassifier(n_estimators=30,random_state=51)
model.fit(X_train, Y_train)
prediction=model.predict(X_test)
print('Accuracy for rbf RandomForestClassifier is ',np.mean(cross_val_score(model, train, train_df['Survived'], cv=3)))

In [ ]:
sns.heatmap(confusion_matrix(prediction,Y_test),annot=True,fmt='2.0f')

In [ ]:
model=svm.SVC(kernel='linear',C=0.1,gamma=0.1,random_state=51)
model.fit(X_train,Y_train)
prediction=model.predict(X_test)
print('Accuracy for linear SVM is',np.mean(cross_val_score(model, train, train_df['Survived'], cv=3)))

In [ ]:
sns.heatmap(confusion_matrix(prediction,Y_test),annot=True,fmt='2.0f')

In [ ]:
model=LinearSVC(random_state=51)
model.fit(X_train, Y_train)
prediction=model.predict(X_test)
print('The accuracy of the NaiveBayes is',np.mean(cross_val_score(model, train, train_df['Survived'], cv=3)))

In [ ]:
sns.heatmap(confusion_matrix(prediction,Y_test),annot=True,fmt='2.0f')

In my opinion, I think since the distribution survived and dead is pretty clear. Therefore, the SVM algorithm can have a good prediction. However, since there are still some exception, the improvement of the prediction would be limited. In my case, the maximum of my SVM model is 78% accuracy. 

About the knowledge of these algorithms, you can refer to this [article](https://towardsdatascience.com/10-machine-learning-algorithms-you-need-to-know-77fb0055fe0).

In [ ]:
model=RandomForestClassifier(n_estimators=30)
model.fit(X_train, Y_train)
prediction = model.predict(test)
submission = pd.read_csv('../input/gender_submission.csv')
submission.Survived = prediction
submission.to_csv('submission.csv',index=False)

**Refer to :**

https://www.kaggle.com/ash316/eda-to-prediction-dietanic

https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy

https://www.kaggle.com/startupsci/titanic-data-science-solutions

https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python


Thanks for sharing

<font size=5>Upvote if you find this helpful. </font>

<font size=5>If you are still in novice tier, I would appreicate that you fill up all the information and become a competition contributor for me! </font>

<font size=5>Thanks</font>